# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [2]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
c.execute("""SELECT contactfirstname, contactlastname, phone, addressline1, creditlimit 
                FROM customers
                WHERE creditlimit > 25000""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Jean,King,7025551838,8489 Strong St.,71800
1,Peter,Ferguson,03 9520 4555,636 St Kilda Road,117300
2,Janine,Labrune,40.67.8555,"67, rue des Cinquante Otages",118200
3,Jonas,Bergulfsen,07-98 9555,Erling Skakkes gate 78,81700
4,Susan,Nelson,4155551450,5677 Strong St.,210500
...,...,...,...,...,...
89,Rosa,Salazar,2155559857,11328 Douglas Av.,72600
90,Sue,Taylor,4155554312,2793 Furth Circle,60300
91,Thomas,Smith,(171) 555-7555,120 Hanover Sq.,43300
92,Valarie,Franco,6175552555,6251 Ingle Ln.,85100


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [13]:
q = """SELECT state, AVG(creditlimit) FROM customers GROUP BY state"""
df = pd.DataFrame(c.execute(q).fetchall())
df.columns = [x[0] for x in c.description]
df

,state,AVG(creditlimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [11]:
q = """SELECT customername, customernumber, ordernumber, status, quantityordered
        FROM customers
        JOIN
        (
            SELECT *
            FROM orders
            JOIN orderdetails
            USING (ordernumber)
        )
        USING (customernumber)
        """
df = pd.DataFrame(c.execute(q).fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,46
2,Atelier graphique,103,10123,Shipped,34
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,39
...,...,...,...,...,...
2991,Kelly's Gift Shop,496,10399,Shipped,29
2992,Kelly's Gift Shop,496,10399,Shipped,30
2993,Kelly's Gift Shop,496,10399,Shipped,57
2994,Kelly's Gift Shop,496,10399,Shipped,58


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [27]:
q = """SELECT customername, customernumber, productname, productcode, SUM(quantityordered) totalordered
        FROM 
        (
            SELECT *
            FROM customers
            JOIN orders
            USING (customernumber)
        )
        JOIN
        (
            SELECT *
            FROM orderdetails
            JOIN products
            USING (productcode)
        )
        USING (ordernumber)
        GROUP BY customername, productname
        HAVING totalordered >= 10 
        ORDER BY TotalOrdered DESC
        """
df = pd.DataFrame(c.execute(q).fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,productName,productCode,totalordered
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174
...,...,...,...,...,...
2526,The Sharp Gifts Warehouse,450,1969 Chevrolet Camaro Z28,S24_3191,13
2527,La Rochelle Gifts,119,1954 Greyhound Scenicruiser,S32_2509,11
2528,Tekni Collectables Inc.,328,American Airlines: B767-300,S700_1691,11
2529,"Extreme Desk Decorations, Ltd",412,1961 Chevrolet Impala,S24_4620,10


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [30]:
q = """SELECT firstname, lastname, employeenumber, officecode
        FROM employees
        WHERE officecode IN (SELECT officecode FROM offices)"""
df = pd.DataFrame(c.execute(q).fetchall())
df.columns = [x[0] for x in c.description]
df

,firstName,lastName,employeeNumber,officeCode
0,Diane,Murphy,1002,1
1,Mary,Patterson,1056,1
2,Jeff,Firrelli,1076,1
3,William,Patterson,1088,6
4,Gerard,Bondur,1102,4
5,Anthony,Bow,1143,1
6,Leslie,Jennings,1165,1
7,Leslie,Thompson,1166,1
8,Julie,Firrelli,1188,2
9,Steve,Patterson,1216,2


In [52]:
q = """SELECT lastname, firstname, employeenumber, officecode
        FROM employees
        WHERE officecode IN 
            (
            SELECT officecode
            FROM employees
            JOIN offices
            USING (officecode)
            GROUP BY 1
            HAVING COUNT(officecode) < 5
            )"""
df = pd.DataFrame(c.execute(q).fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,lastName,firstName,employeeNumber,officeCode
0,Patterson,William,1088,6
1,Firrelli,Julie,1188,2
2,Patterson,Steve,1216,2
3,Tseng,Foon Yue,1286,3
4,Vanauf,George,1323,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!